In [14]:
# Celda Definitiva con la función del Método QR

using LinearAlgebra

"""
    metodo_qr_final(a_diag, b_subdiag; tol=1e-14, max_iter_total=300)

Calcula todos los eigenvalores de una matriz simétrica tridiagonal.
Esta es una implementación robusta del algoritmo QR con shifts de Wilkinson.

# Argumentos
- `a_diag::Vector{Float64}`: La diagonal principal.
- `b_subdiag::Vector{Float64}`: La subdiagonal (longitud n-1).

# Retorna
- `Vector{Float64}`: Un vector con todos los eigenvalores.
"""
function metodo_qr_final(a_diag::Vector{Float64}, b_subdiag::Vector{Float64}; 
                           tol::Float64=1e-14, max_iter_total::Int=300)
    
    a = copy(a_diag)
    b = copy(b_subdiag)
    n = length(a)
    eigenvalores = Float64[]
    
    iter_global = 0

    # Bucle principal que se reduce con cada eigenvalor encontrado (deflación)
    while n > 0
        if iter_global > max_iter_total
            error("El algoritmo no convergió en el número máximo de iteraciones totales.")
        end

        # Bucle de iteraciones QR para el subproblema actual de tamaño n
        for iter_local in 1:n # Típicamente converge rápido
            iter_global += 1

            # 1. Búsqueda de deflación: si b[n-1] es pequeño, a[n] es un eigenvalor
            if n == 1 || abs(b[n-1]) <= tol * (abs(a[n-1]) + abs(a[n]))
                push!(eigenvalores, a[n])
                n -= 1 # Reducimos el tamaño del problema
                break  # Salimos del bucle de iteraciones local
            end

            # 2. Calcular el shift de Wilkinson (para la submatriz actual)
            g = (a[n-1] - a[n]) / (2 * b[n-1])
            r = hypot(g, 1.0)
            sigma = a[n] - (b[n-1] / (g + sign(g) * r))

            # 3. Paso QR implícito (bucle de rotaciones de Givens)
            # Este es el núcleo del algoritmo, ahora implementado de forma robusta.
            c = 1.0
            s = 0.0
            p = 0.0

            for j in 1:(n-1)
                f = s * b[j]
                g = c * b[j]

                # Nueva rotación
                r = hypot(a[j] - sigma, f)
                c_new = (a[j] - sigma) / r
                s_new = f / r
                
                if j != 1
                    b[j-1] = r
                end

                a[j] = c * c_new * (a[j] - sigma) + s * c_new * f + p
                p = s_new * (c*f + s*(a[j+1] - sigma))
                a[j+1] = c*c*(a[j+1] - sigma) - s*s_new*f + a[j] - p
                
                c = c_new
                s = s_new
            end

            b[n-1] = p
            a[n] = a[n] - p
            
        end # Fin del bucle de iteraciones local
    end # Fin del bucle principal
    
    return eigenvalores
end

# --- EJEMPLO DE USO (igual que antes) ---

# Matriz tridiagonal simétrica T del ejercicio de Householder
a_diagonal = [4.0, 3.3333333333333335, -0.5238095238095237, 1.1904761904761905]
b_subdiagonal = [-3.0, -0.6666666666666665, -0.19047619047619048]

println("--- Calculando Eigenvalores con el Método QR (Versión Definitiva) ---")
eigen_calculados = metodo_qr_final(a_diagonal, b_subdiagonal)

println("\nEigenvalores calculados:")
display(sort(eigen_calculados))

# --- Verificación ---
T_completa = Tridiagonal(b_subdiagonal, a_diagonal, b_subdiagonal)
eigen_reales = eigvals(T_completa)

println("\nEigenvalores de referencia (función `eigvals`):")
display(sort(eigen_reales))

if isapprox(sort(eigen_calculados), sort(eigen_reales))
    println("\n✅ Verificación exitosa: Los eigenvalores coinciden.")
else
    println("\n❌ Error en la verificación.")
end

--- Calculando Eigenvalores con el Método QR (Versión Definitiva) ---

Eigenvalores calculados:


4-element Vector{Float64}:
 0.7316888815469216
 2.556170374899957
 4.954375666763851
 6.219513919966206


Eigenvalores de referencia (función `eigvals`):


4-element Vector{Float64}:
 -0.7461216087086284
  0.8160241895808965
  1.217471880409708
  6.712625538718026


❌ Error en la verificación.
